# Athens Neighborhoods K-mean clustering

### Part 1

Opening a New Shopping Mall in Athens, Greece

Build a dataframe of neighborhoods in Athens, Greece by web scraping the data from Wikipedia page
Get the geographical coordinates of the neighborhoods
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighborhoods
Select the best cluster to open a new shopping mall

In [6]:
#Import libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 94 kB 6.0 MB/s  eta 0:00:01
Libraries imported.


In [8]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Athens").text
soup = BeautifulSoup(data, 'html.parser')

# create a list to store neighborhood data
neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

# create a new DataFrame from the list
at_df = pd.DataFrame({"Neighborhood": neighborhoodList})

at_df.head()

,Neighborhood
0,"Aerides, Athens"
1,"Agios Eleftherios, Athens"
2,"Agios Panteleimonas, Athens"
3,Akadimia Platonos
4,"Akadimia, Athens"


In [20]:
# print the number of rows of the dataframe
at_df.shape
num_neigh=at_df.shape[0]
print("Athens has {} neighborhoods".format(num_neigh))

Athens has 63 neighborhoods


In [21]:
at_df

,Neighborhood
0,"Aerides, Athens"
1,"Agios Eleftherios, Athens"
2,"Agios Panteleimonas, Athens"
3,Akadimia Platonos
4,"Akadimia, Athens"
5,"Ampelokipoi, Athens"
6,Anafiotika
7,Ano Petralona
8,Asteroskopeio
9,"Asyrmatos, Athens"


In [27]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Athens, Greece'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [29]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in at_df["Neighborhood"].tolist() ]
coords

[[37.97614000000004, 23.73640000000006],
 [38.01997000000006, 23.726270000000056],
 [37.996830000000045, 23.726950000000045],
 [37.990950000000055, 23.70718000000005],
 [37.98675000000003, 23.71103000000005],
 [37.99106000000006, 23.764290000000074],
 [37.972240000000056, 23.72852000000006],
 [37.96948000000003, 23.70897000000008],
 [37.97438000000005, 23.719200000000058],
 [37.9316627308821, 23.721171420020955],
 [38.00727000000006, 23.728040000000078],
 [37.96667000000008, 23.669400000000053],
 [37.99793000000005, 23.775160000000028],
 [37.992680349285855, 23.767630320154506],
 [37.977670000000046, 23.747780000000034],
 [37.988170000000025, 23.733610000000056],
 [37.97872000000007, 23.714260000000024],
 [37.99711000000008, 23.76406000000003],
 [37.982920000000036, 23.770980000000066],
 [37.95998000000003, 23.740830000000074],
 [38.02047498933519, 23.84525292476216],
 [37.976410000000044, 23.755570000000034],
 [37.96691000000004, 23.74022000000008],
 [37.96691000000004, 23.74022000000

In [30]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [31]:
# merge the coordinates into the original dataframe
at_df['Latitude'] = df_coords['Latitude']
at_df['Longitude'] = df_coords['Longitude']

In [36]:

# check the neighborhoods and the coordinates
print(at_df.shape)
at_df
# save the DataFrame as CSV file
#at_df.to_csv("at_df.csv", index=False)

(63, 3)


### Part 2. Create a map of Athens, Greece with neighborhoods superimposed on top

In [40]:
# get the coordinates of Kuala Lumpur
address = 'Athens, Greece'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Athens, Greece {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Athens, Greece 37.9839412, 23.7283052.


In [43]:

# create map of Toronto using latitude and longitude values
map_at = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(at_df['Latitude'], at_df['Longitude'], at_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_at)  
    
map_at

In [44]:
# save the map as HTML file
map_at.save('map_at.html')

### Part 3 Use Frousquare API to explore neighborhoods in Athens

In [45]:
CLIENT_ID = 'HMOFRMIWKFN5PEO5EMRXFO5H40JF3FNEWZJHDA2Y4NVM44IE' # your Foursquare ID
CLIENT_SECRET = 'KHZOTSX0W2SRUB5AL2ARNCMHE2J2G3M2R1RJBPFJJBADMYVD' # your Foursquare Secret
ACCESS_TOKEN = 'CWI30Y2SFNZRSXUPJN334LPZTY0US5FVIPKRK0GHQGFHQ3A5' # your FourSquare Access Token
VERSION = '20210302'
LIMIT = 30

In [47]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(at_df['Latitude'], at_df['Longitude'], at_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [48]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6285, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Aerides, Athens",37.97614,23.7364,Hotel Grande Bretagne,37.976178,23.735312,Hotel
1,"Aerides, Athens",37.97614,23.7364,Louis Vuitton,37.977704,23.736446,Boutique
2,"Aerides, Athens",37.97614,23.7364,GB Roof Garden Restaurant,37.976116,23.735301,Roof Deck
3,"Aerides, Athens",37.97614,23.7364,Carpo,37.977183,23.738547,Gourmet Shop
4,"Aerides, Athens",37.97614,23.7364,Toyroom Athens,37.977593,23.735721,Nightclub


In [54]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Aerides, Athens",100,100,100,100,100,100
"Agios Eleftherios, Athens",100,100,100,100,100,100
"Agios Panteleimonas, Athens",100,100,100,100,100,100
Akadimia Platonos,100,100,100,100,100,100
"Akadimia, Athens",100,100,100,100,100,100
"Ampelokipoi, Athens",100,100,100,100,100,100
Anafiotika,100,100,100,100,100,100
Ano Petralona,100,100,100,100,100,100
Asteroskopeio,100,100,100,100,100,100


In [50]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 197 uniques categories.


In [51]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Hotel', 'Boutique', 'Roof Deck', 'Gourmet Shop', 'Nightclub',
       'Park', 'Coffee Shop', 'Bookstore', 'Ice Cream Shop', 'Plaza',
       'Sushi Restaurant', 'Shopping Mall', 'Bagel Shop', 'Museum',
       'Dessert Shop', 'Department Store', 'Café', 'Pie Shop', 'Bar',
       'Wine Shop', 'Wine Bar', 'Comfort Food Restaurant',
       'History Museum', 'American Restaurant', 'Chocolate Shop',
       'Italian Restaurant', 'Jewelry Store', 'Souvlaki Shop',
       'Fast Food Restaurant', 'Cupcake Shop', 'Garden', 'Whisky Bar',
       'Indie Movie Theater', 'Magirio', "Women's Store", 'Cocktail Bar',
       'Gift Shop', 'Clothing Store', 'Middle Eastern Restaurant',
       'Cheese Shop', 'Falafel Restaurant', 'Burger Joint',
       'Historic Site', 'Tennis Court', 'Art Museum', 'Track Stadium',
       'Greek Restaurant', 'Pedestrian Plaza', 'Mexican Restaurant',
       'Hill'], dtype=object)

In [59]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### Part 4. Analyze Neighborhoods

In [60]:

# one hot encoding
at_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
at_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [at_onehot.columns[-1]] + list(at_onehot.columns[:-1])
at_onehot = at_onehot[fixed_columns]

print(at_onehot.shape)
at_onehot.head()

(6285, 198)


,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Bougatsa Shop,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Butcher,Café,Camera Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Department Store,Dessert Shop,Dog Run,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fish Taverna,Food & Drink Shop,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nightclub,Optical Shop,Other Great Outdoors,Other Nightlife,Ouzeri,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Roof Deck,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Social Club,Souvlaki Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Trail,Tsipouro Restaurant,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Aerides, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Aerides, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Aerides, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Aerides, A

In [61]:
at_grouped = at_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(at_grouped.shape)
at_grouped

(63, 198)


,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Bougatsa Shop,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Butcher,Café,Camera Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Department Store,Dessert Shop,Dog Run,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fish Taverna,Food & Drink Shop,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nightclub,Optical Shop,Other Great Outdoors,Other Nightlife,Ouzeri,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Roof Deck,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Social Club,Souvlaki Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Trail,Tsipouro Restaurant,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Aerides, Athens",0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.01,0.000000,0.040000,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.000000,0.04,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,0.00,0.130000,0.00,0.01,0.00,0.01,0.01,0.01,0.090000,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.050000,0.00,0.000000,0.000000,0.00,0.00,0.01,0.00,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.04,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.01,0.04,0.01,0.00,0.01,0.00,0.020000,0.00,0.01,0.00,0.020000,0.01,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.010000,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.010000,0.01,0.01,0.00,0.00,0.000000,0.01,0.000000,0.00,0.00,0.060000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.000000,0.000000,0.00,0.00,0.010000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.01,0.000000,0.00,0.000000,0.00,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.03,0.01,0.01,0.000000
1,"Agios Eleftherios, Athens",0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.

In [62]:

len(at_grouped[at_grouped["Shopping Mall"] > 0])

18

### Create a new DataFrame for Shopping Mall data only

In [63]:
at_mall = at_grouped[["Neighborhoods","Shopping Mall"]]

In [64]:
at_mall.head()

,Neighborhoods,Shopping Mall
0,"Aerides, Athens",0.01
1,"Agios Eleftherios, Athens",0.00
2,"Agios Panteleimonas, Athens",0.00
3,Akadimia Platonos,0.00
4,"Akadimia, Athens",0.00


### Cluster neighborhoods

Run k-means to cluster the neighborhoods in Athens into 2 clusters.

In [66]:
# set number of clusters
kclusters = 2

at_clustering = at_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(at_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [68]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
at_merged = at_mall.copy()

# add clustering labels
at_merged["Cluster Labels"] = kmeans.labels_

In [69]:
at_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
at_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"Aerides, Athens",0.01,1
1,"Agios Eleftherios, Athens",0.00,0
2,"Agios Panteleimonas, Athens",0.00,0
3,Akadimia Platonos,0.00,0
4,"Akadimia, Athens",0.00,0


In [70]:
#merge at_grouped w/ at_data to aadd latitued & longitude or each neighborhood
at_merged = at_merged.join(at_df.set_index("Neighborhood"), on="Neighborhood")

print(at_merged.shape)
at_merged.head() # check the last columns!

(63, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Aerides, Athens",0.01,1,37.97614,23.73640
1,"Agios Eleftherios, Athens",0.00,0,38.01997,23.72627
2,"Agios Panteleimonas, Athens",0.00,0,37.99683,23.72695
3,Akadimia Platonos,0.00,0,37.99095,23.70718
4,"Akadimia, Athens",0.00,0,37.98675,23.71103


In [71]:
# sort the results by Cluster Labels
print(at_merged.shape)
at_merged.sort_values(["Cluster Labels"], inplace=True)
at_merged

(63, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
31,Kypriadou,0.00,0,37.993974,23.745041
28,Koukaki,0.00,0,37.963720,23.723530
30,Kynosargous,0.00,0,38.023995,23.673820
32,"Kypseli, Athens",0.00,0,38.003280,23.740600
33,"Makrygianni, Athens",0.00,0,37.968070,23.726870
34,Metaxourgeio,0.00,0,37.983040,23.718690
35,"Mets, Athens",0.00,0,37.965220,23.735810
38,Nea Filothei,0.00,0,37.996820,23.763110
40,"Neos Kosmos, Athens",0.00,0,37.961420,23.728710
42,Pangrati,0.00,0,37.969730,23.747300


### Finally, let's visualize the resulting clusters

In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(at_merged['Latitude'], at_merged['Longitude'], at_merged['Neighborhood'], at_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters

In [75]:
at_merged.loc[at_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
31,Kypriadou,0.0,0,37.993974,23.745041
28,Koukaki,0.0,0,37.963720,23.723530
30,Kynosargous,0.0,0,38.023995,23.673820
32,"Kypseli, Athens",0.0,0,38.003280,23.740600
33,"Makrygianni, Athens",0.0,0,37.968070,23.726870
34,Metaxourgeio,0.0,0,37.983040,23.718690
35,"Mets, Athens",0.0,0,37.965220,23.735810
38,Nea Filothei,0.0,0,37.996820,23.763110
40,"Neos Kosmos, Athens",0.0,0,37.961420,23.728710
42,Pangrati,0.0,0,37.969730,23.747300


### Cluster 1

In [76]:
at_merged.loc[at_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
15,Exarcheia,0.01,1,37.988170,23.733610
60,Treis Gefyres,0.01,1,37.976140,23.736400
26,Kolonaki,0.01,1,37.977620,23.743590
57,Template:Neighbourhoods in Athens,0.01,1,37.976140,23.736400
56,Skouze Hill,0.01,1,37.977295,23.728683
29,Kountouriotika,0.01,1,37.988560,23.735324
52,Psyri,0.01,1,37.978760,23.726670
61,"Vathi, Athens",0.01,1,37.987166,23.726195
37,Mount Lycabettus,0.01,1,37.978000,23.736560
49,Probonas,0.01,1,37.976140,23.736400


As you can tell from the Folium map, the neighborhoods with Shopping malls (purple dots) are clustered around downtown Athens. The majority of the neighborhoods that are further from the downtown do not have shopping malls. There is a lot of potential at those areas.